Intersect news with Company ontology to proof if there are newses that corresponds to the companies that does not exist. <BR>
Those newses will go into exception\n

In [1]:


import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
''
sc = pyspark.SparkContext(appName="company_news_stock_price_ontology")
spark = SparkSession(sc)
from pyspark.sql.types import StructField, StructType, StringType, LongType,DoubleType,ArrayType,FloatType, BooleanType
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
import numpy as np

In [2]:
from skainet_spark import Pipeline, transform, Input, Output, Metadata,ValidatedPipeline,print_statistics

In [3]:
@transform(spark,
    news_ontology = Input('/news/processed/news_processed.parquet', spark),
    company_ontology = Input('/company_details/processed/company_details_processed.parquet', spark),
    stock_clean = Input('/yahoo_stock_price/clean/yahoo_price_clean.parquet',spark),
           
    metadata_news = Metadata('/news/ontology/news_metadata_ontology.csv',spark),
    metadata_comp_det = Metadata('/company_details/ontology/company_details_metadata_ontology.csv',spark),
    metadata_stock = Metadata('/stock_price/ontology/stock_price_metadata_ontology.csv',spark),
    
    path_news = Output('/news/ontology/news_ontology.parquet'),
    path_news_excp = Output('/news/exception/news_ontology_exception.parquet'),
           
    path_comp = Output('/company_details/ontology/company_details_ontology.parquet'),
    path_comp_excp = Output('/company_details/exception/company_details_ontology_exception.parquet'),
           
    path_stock = Output('/stock_price/ontology/stock_price_ontology.parquet'),
    path_stock_excep = Output('/stock_price/exception/stock_price_ontology_exception.parquet'),
)
def synergize_ontology(spark, news_ontology, company_ontology, stock_clean,
                      metadata_news,metadata_comp_det,metadata_stock,path_news,path_news_excp, 
                      path_comp,path_comp_excp, path_stock,path_stock_excep
                      ):
    pipel_news = Pipeline(news_ontology)
    pipel_comp = Pipeline(company_ontology)
    pipe_stock = Pipeline(stock_clean)
    
        #debug BRK-A
    pipel_comp.dataframe.select('ticker_symbol').where(F.col('ticker_symbol') == F.lit('UBER')).show(1,0)
    #
    
    print('Company Details')
    print_statistics(pipel_comp)
    print('News Content')
    print_statistics(pipel_news)

    print('DISCARD NEWS THAT DO NOT MENTION TICKER FROM COMPANIES DETAIL')
    join_on = pipel_comp.dataframe['ticker_symbol']==pipel_news.dataframe['mentioned_tickers']
    
    pipe_news_filtered = pipel_news.join(pipel_comp,join_on,'left_semi')
    print_statistics(pipe_news_filtered)
    print(pipe_news_filtered.dataframe.where(F.col('mentioned_tickers').isNull()).count())
    
    print('DISCARD STOCKS THAT DO NOT MENTION TICKER FROM COMPANIES DETAIL')
    join_on = pipe_stock.dataframe['ticker_symbol']==pipel_comp.dataframe['ticker_symbol']
    print_statistics(pipe_stock)
    pipe_stocks_filtered = pipe_stock.join(pipel_comp,join_on,'left_semi')
    print_statistics(pipe_stocks_filtered)
    print(pipe_stocks_filtered.dataframe.count())
    
    print('SAVE NEWS')
    
    schema = metadata_news()
    validated_pipe = ValidatedPipeline(pipe_news_filtered, schema)
    validated_pipe = (validated_pipe
                      .validate()
                     )
    
    validated_pipe.write(path_news,path_news_excp)
    
    print('SAVE STOCKS')
    
    schema = metadata_stock()
    validated_pipe = ValidatedPipeline(pipe_stocks_filtered, schema)
    validated_pipe = (validated_pipe
                      .validate()
                     )
    
    validated_pipe.write(path_stock,path_stock_excep)
    
    print('SAVE COMPANIES')
    
    schema = metadata_comp_det()
    validated_pipe = ValidatedPipeline(pipel_comp, schema)
    validated_pipe = (validated_pipe
                      .validate()
                     )
    
    validated_pipe.write(path_comp,path_comp_excp)

In [4]:
synergize_ontology(spark)

+-------------+
|ticker_symbol|
+-------------+
+-------------+

Company Details
root
 |-- ticker_symbol: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- website_link: string (nullable = true)
 |-- num_employees: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- telephone: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- shareholders_name: string (nullable = true)
 |-- percentage: double (nullable = true)

cols: 13 rows: 6612
News Content
root
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)
 |-- published_date: date (nullable = true)
 |-- publisher: string (nullable = true)
 |-- mentioned_tickers: string (nullable = true)
 |-- news_id: string (nullable = true)

cols: 6 rows: 7323
DISCARD NEWS THAT DO NOT MENTION TICKER FROM COMPANIES DETAIL
